## Pre-requisites

### Load the dataset

In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv", header=True, inferSchema=True)
df.display()

### Transaction 01-- Write data into delta format

In [0]:
df.write.format("delta").save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")

### Transaction 02-- Overwrite delta folder

In [0]:
from pyspark.sql.functions import col
df.filter(col("country")=="India").write.mode("overwrite").save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")

### Reads the latest transactions

In [0]:
spark.read.load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").display()

In [0]:
spark.read.option('versionAsOf', 0).load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").display()

### Read transaction logs

#### Approach 01

In [0]:
spark.read.load(format='text', path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/_delta_log/00000000000000000000.json").display()

In [0]:
spark.read.load(format='text', path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/_delta_log/00000000000000000001.json").display()

#### Approach 2

In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/")
delta_table.history().display()

### Transaction 03-- Updates are allowed

In [0]:
df_delta = spark.read.load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/"
)
df_delta.createOrReplaceTempView("user_vw")

In [0]:
%sql
 
UPDATE user_vw
SET country='Bharat'
where country='India'
 

In [0]:
spark.read.load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").display()